I utilized Google OR-Tools to tackle a series of linear programming exercises. Initially, I aimed to devise a universal function to systematically address these exercises, hoping to consolidate the process of defining and solving each distinct problem within a singular, flexible framework.

Regrettably, this approach met with difficulties (algorithmic : "too many values to unpack" while using tuples for contraints). It became apparent that the unique subtleties of each LP problem defied a one-size-fits-all solution. 

Particularly, exercises 6 and 7 highlighted infeasible scenarios—conditions where constraints formed an unsolvable problem space. Comprehending the solver's feedback, such as 'infeasible' or 'unbounded', was imperative for these exercises.

Confronted with these challenges, I shifted to a customized approach for each case. I meticulously defined constraints, objective functions, and variable bounds before engaging OR-Tools for resolution. This method offered a granular level of control and a deeper dive into each problem's architecture.

**Found help here :**

https://github.com/google/or-tools/blob/stable/ortools/linear_solver/linear_solver.h#L442

https://github.com/google/or-tools/blob/a4a3d4a786fae1d6b7f1fdbc97d4c4f363212ef1/ortools/linear_solver/python/linear_solver.i#L284-L291

In [1]:
from ortools.linear_solver import pywraplp

## **Sketch of the first idea**

In [6]:
# Function to create the LP solver
def create_solver(name='SCIP'):
    return pywraplp.Solver.CreateSolver(name)

In [7]:
# Function to solve an LP problem and print the solution
def solve_lp_problem(solver, objective_coefficients, constraints, bounds, maximize=True):
    # Variables
    x = [solver.NumVar(lb, ub, f'x{i}') for i, (lb, ub) in enumerate(bounds)]

    # Objective
    objective = solver.Objective()
    for i, coeff in enumerate(objective_coefficients):
        objective.SetCoefficient(x[i], coeff)
    if maximize:
        objective.SetMaximization()
    else:
        objective.SetMinimization()

    # Constraints
    for lhs, rhs in constraints:
        constraint = solver.RowConstraint(lhs, rhs, '')
        for i in range(len(x)):
            constraint.SetCoefficient(x[i], lhs[i])

    # Solve
    status = solver.Solve()

    # Print solution
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found:')
        for i, var in enumerate(x):
            print(f'Variable x{i+1} = {var.solution_value()}')
        print('Objective value =', solver.Objective().Value())
    else:
        if status == pywraplp.Solver.INFEASIBLE:
            print('No feasible solution found.')
        else:
            print('No solution found.')

## **Second Idea and solutions**

In [ ]:
solver = create_solver()
solve_lp_problem(
    solver=solver,
    coefficients=[22, 28],
    constraint_lhs=[[8, 10], [2, 3]],
    constraint_rhs=[3400, 960],
    bounds=[(0, solver.infinity())] * 2,
    objective_max=True
)

## **Exercise 1:** *Router Production Optimization*


In this scenario, we have to analyze a computer system components manufacturer focusing on the assembly of two wireless router models: A and B. The production is constrained by the availability of materials and labor. Model A yields a profit of $22 per unit, while Model B brings $28 per unit. We have to optimize the production to maximize profits, given unlimited market demand.


In [34]:
def solve_exercise_1():
    solver = create_solver()

    # Variables x_A and x_B
    x_A = solver.NumVar(0, solver.infinity(), 'x_A')
    x_B = solver.NumVar(0, solver.infinity(), 'x_B')
    
    # Objective Function: Maximize 22x_A + 28x_B
    objective = solver.Objective()
    objective.SetCoefficient(x_A, 22)
    objective.SetCoefficient(x_B, 28)
    objective.SetMaximization()
    
    # Constraints
    # 8x_A + 10x_B ≤ 3400
    material_constraint = solver.Constraint(-solver.infinity(), 3400)
    material_constraint.SetCoefficient(x_A, 8)
    material_constraint.SetCoefficient(x_B, 10)
    
    # 2x_A + 3x_B ≤ 960
    labor_constraint = solver.Constraint(-solver.infinity(), 960)
    labor_constraint.SetCoefficient(x_A, 2)
    labor_constraint.SetCoefficient(x_B, 3)
    
    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution or status
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        print('x_A =', x_A.solution_value())
        print('x_B =', x_B.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found. The problem is infeasible.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print(f'Solver status is {status}: Problem could not be solved due to other reasons.')


solve_exercise_1()


Optimal Solution Found:
x_A = 150.0000000000001
x_B = 219.99999999999997
Objective value = 9460.000000000004


## **Exercise 2 :** *minimizing combined production costs*

Exercise 2 presents an inventory and production problem. We're trying to minimize the total costs, considering production and storage costs, given constraints on production capacity and inventory levels over a four-week period.

The objective is to minimize the cost function:

z = 15(x1 + x2 + x3 + x4) + 3(A1 + A2 + A3)

the x are the number of units produced during week, and the 'A' are the number of items remaining at the end of each week.(everything is linear the obj function and the contraints)

In [35]:
def solve_exercise_2():
    solver = create_solver()

    # Variables for units produced each week
    x = [solver.NumVar(500, 700, f'x{i}') for i in range(1, 5)]
    
    # Variables for inventory at the end of each week, not decision variables, but derived from x
    A = [solver.NumVar(0, solver.infinity(), f'A{i}') for i in range(1, 5)]

    # Objective Function: Minimize production and storage costs
    objective = solver.Objective()
    for xi in x:
        objective.SetCoefficient(xi, 15)  # Production cost per unit
    for Ai in A[:-1]:  # Exclude A4 because there is no cost for ending inventory in the last week
        objective.SetCoefficient(Ai, 3)  # Storage cost per unit
    objective.SetMinimization()
    
    # Inventory balance constraints (A[i] = A[i-1] + x[i] - demand[i])
    initial_inventory = 250
    demands = [900, 600, 800, 600]  # Given demand for each week
    for i in range(4):
        if i == 0:
            solver.Add(A[i] == initial_inventory + x[i] - demands[i])
        else:
            solver.Add(A[i] == A[i-1] + x[i] - demands[i])

    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution or status
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        for i in range(4):
            print(f'x{i+1} =', x[i].solution_value())
            print(f'A{i+1} =', A[i].solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found. The problem is infeasible.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print(f'Solver status is {status}: Problem could not be solved due to other reasons.')


solve_exercise_2()


Optimal Solution Found:
x1 = 650.0
A1 = 0.0
x2 = 700.0
A2 = 100.0
x3 = 700.0
A3 = 0.0
x4 = 600.0
A4 = 0.0
Objective value = 40050.0


## **Exercise 3 :** *Unique optimal feasible solution I*

In [27]:
def solve_exercise_3():
    solver = create_solver()
    
    # Variables x1 and x2
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    
    # Objective Function: Maximize 3x1 + x2
    objective = solver.Objective()
    objective.SetCoefficient(x1, 3)
    objective.SetCoefficient(x2, 1)
    objective.SetMaximization()
    
    # Constraints
    # x2 <= 5
    constraint1 = solver.Constraint(-solver.infinity(), 5)
    constraint1.SetCoefficient(x2, 1)
    
    # x1 + x2 <= 10
    constraint2 = solver.Constraint(-solver.infinity(), 10)
    constraint2.SetCoefficient(x1, 1)
    constraint2.SetCoefficient(x2, 1)
    
    # -x1 + x2 >= -2 (which we convert to -x1 + x2 <= 2 for OR-Tools)
    constraint3 = solver.Constraint(-2, solver.infinity())
    constraint3.SetCoefficient(x1, -1)
    constraint3.SetCoefficient(x2, 1)
    
    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution Found:')
        print('x1 =', x1.solution_value())
        print('x2 =', x2.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found.')
    else:
        print('Solver status:', status)
        

solve_exercise_3()



Solution Found:
x1 = 6.0
x2 = 4.0
Objective value = 22.0


## **Exercise 4 :** *Unique optimal feasible solution*

In [28]:
def solve_exercise_4():
    solver = create_solver()

    # Variables x1 and x2
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    
    # Objective Function: Minimize x1 + x2
    objective = solver.Objective()
    objective.SetCoefficient(x1, 1)
    objective.SetCoefficient(x2, 1)
    objective.SetMinimization()
    
    # Constraints
    # 3x1 + x2 >= 6
    constraint1 = solver.Constraint(6, solver.infinity())
    constraint1.SetCoefficient(x1, 3)
    constraint1.SetCoefficient(x2, 1)
    
    # x2 >= 3
    constraint2 = solver.Constraint(3, solver.infinity())
    constraint2.SetCoefficient(x2, 1)
    
    # x1 <= 4
    constraint3 = solver.Constraint(-solver.infinity(), 4)
    constraint3.SetCoefficient(x1, 1)
    
    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution Found:')
        print('x1 =', x1.solution_value())
        print('x2 =', x2.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found.')
    else:
        print('Solver status:', status)


solve_exercise_4()


Solution Found:
x1 = 1.0
x2 = 3.0
Objective value = 4.0


## **Exercise 5 :** *Multiple optimal feasible solutions*

In [30]:
def solve_exercise_5():
    solver = create_solver()

    # Variables x1 and x2
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    
    # Objective Function: Maximize x1 + 2x2
    objective = solver.Objective()
    objective.SetCoefficient(x1, 1)
    objective.SetCoefficient(x2, 2)
    objective.SetMaximization()
    
    # Constraints
    # -x1 + x2 <= 2
    constraint1 = solver.Constraint(-solver.infinity(), 2)
    constraint1.SetCoefficient(x1, -1)
    constraint1.SetCoefficient(x2, 1)
    
    # x1 + 2x2 <= 8
    constraint2 = solver.Constraint(-solver.infinity(), 8)
    constraint2.SetCoefficient(x1, 1)
    constraint2.SetCoefficient(x2, 2)
    
    # x1 <= 6
    constraint3 = solver.Constraint(-solver.infinity(), 6)
    constraint3.SetCoefficient(x1, 1)
    
    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution
    if status == pywraplp.Solver.OPTIMAL:
        print('One Optimal Solution Found:')
        print('x1 =', x1.solution_value())
        print('x2 =', x2.solution_value())
        print('Objective value =', objective.Value())
        
        # To find another optimal solution on the edge of optimality,
        # I induced a perturbation on the objective function slightly and then resolved.
        objective.SetCoefficient(x1, 1 - 1e-6)
        objective.SetCoefficient(x2, 2)
        solver.Solve()
        print('Another Optimal Solution Found:')
        print('x1 =', x1.solution_value())
        print('x2 =', x2.solution_value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found.')
    else:
        print('Solver status:', status)


solve_exercise_5()


One Optimal Solution Found:
x1 = 6.0
x2 = 1.0
Objective value = 8.0
Another Optimal Solution Found:
x1 = 1.333333333333334
x2 = 3.333333333333333


## **Exercise 6 :** *No optimal feasible solutions*

In [32]:
def solve_exercise_6():
    solver = create_solver()

    # Variables x1 and x2
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    
    # Objective Function: Maximize 3x1 + x2
    objective = solver.Objective()
    objective.SetCoefficient(x1, 3)
    objective.SetCoefficient(x2, 1)
    objective.SetMaximization()
    
    # Constraints
    # x1 + x2 >= 4
    constraint1 = solver.Constraint(4, solver.infinity())
    constraint1.SetCoefficient(x1, 1)
    constraint1.SetCoefficient(x2, 1)
    
    # -x1 + x2 <= 4
    constraint2 = solver.Constraint(-solver.infinity(), 4)
    constraint2.SetCoefficient(x1, -1)
    constraint2.SetCoefficient(x2, 1)
    
    # -x1 + 2x2 >= -4 (rearranged from -x1 + 2x2 >= -4)
    constraint3 = solver.Constraint(-4, solver.infinity())
    constraint3.SetCoefficient(x1, -1)
    constraint3.SetCoefficient(x2, 2)
    
    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found (which is unexpected):')
        print('x1 =', x1.solution_value())
        print('x2 =', x2.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print('Solver status:', status,"The problem has no optimal solution.")


solve_exercise_6()


Solver status: 4 The problem has no optimal solution.


## **Exercise 7 :** *No feasible solutions*

In [33]:
def solve_exercise_7():
    solver = create_solver()

    # Variables x1 and x2
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    
    # Objective Function: Maximize 3x1 + x2
    objective = solver.Objective()
    objective.SetCoefficient(x1, 3)
    objective.SetCoefficient(x2, 1)
    objective.SetMaximization()
    
    # Constraints
    # -x1 + x2 >= 4
    constraint1 = solver.Constraint(4, solver.infinity())
    constraint1.SetCoefficient(x1, -1)
    constraint1.SetCoefficient(x2, 1)
    
    # -x1 + 2x2 <= -4
    constraint2 = solver.Constraint(-solver.infinity(), -4)
    constraint2.SetCoefficient(x1, -1)
    constraint2.SetCoefficient(x2, 2)
    
    # Solve the problem
    status = solver.Solve()
    
    # Check and print the solution or status
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        print('x1 =', x1.solution_value())
        print('x2 =', x2.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found. The problem is infeasible.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print(f'Solver status is {status}: Problem could not be solved due to other reasons.')


solve_exercise_7()


No feasible solution found. The problem is infeasible.
